In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint 

In [2]:
mongo = MongoClient(port=27017)

In [3]:
print(mongo.list_database_names())

['Homework1', 'Project_3', 'admin', 'classDB', 'config', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'uk_food']


In [8]:
db = mongo['Project_3']

In [9]:
print(db.list_collection_names())

['video_games']


In [10]:
video_games = db['video_games']

In [11]:
print(db.video_games.find_one())

{'_id': ObjectId('6583d28ab91a3d86a9e22029'), 'name': 'The Legend of Zelda: Ocarina of Time', 'platform': ' Nintendo 64', 'release_date': '23-Nov-98', 'summary': 'As a young boy, Link is tricked by Ganondorf, the King of the Gerudo Thieves. The evil human uses Link to gain access to the Sacred Realm, where he places his tainted hands on Triforce and transforms the beautiful Hyrulean landscape into a barren wasteland. Link is determined to fix the problems he helped to create, so with the help of Rauru he travels through time gathering the powers of the Seven Sages.', 'user_review': 9.1}


In [12]:
count_tbd = video_games.count_documents({'user_review': 'tbd'})

print(f"Number of documents where user_review is 'tbd': {count_tbd}")

Number of documents where user_review is 'tbd': 1365


In [13]:
# Delete documents with user_review equal to "tbd"
result = video_games.delete_many({'user_review': 'tbd'})
print(f"{result.deleted_count} documents deleted")
# Now, retrieve the remaining documents and perform aggregation
pipeline = [
    {
        '$group': {
            '_id': '$name',
            'mean_user_review': {'$avg': '$user_review'}
        }
    }
]

result_after_deletion = list(video_games.aggregate(pipeline))

for item in result_after_deletion[:5]:
    pprint(item)

1365 documents deleted
{'_id': 'Oddworld: Soulstorm', 'mean_user_review': 6.35}
{'_id': 'Devil May Cry 3: Special Edition', 'mean_user_review': 8.6}
{'_id': "Jumper: Griffin's Story", 'mean_user_review': 3.3}
{'_id': "Cabela's Dangerous Hunts 2011", 'mean_user_review': 6.0}
{'_id': 'Happy Birthdays', 'mean_user_review': 7.2}


In [14]:
documents_with_tbd = video_games.find({'user_review': 'tbd'})

# Print or process the matching documents
for document in documents_with_tbd:
    print(document)

In [15]:
cleaned_df = pd.DataFrame(result_after_deletion)
cleaned_df.head(15)

,_id,mean_user_review
0,Oddworld: Soulstorm,6.350000
1,Devil May Cry 3: Special Edition,8.600000
2,Jumper: Griffin's Story,3.300000
3,Cabela's Dangerous Hunts 2011,6.000000
4,Happy Birthdays,7.200000
5,Fatal Frame II: Crimson Butterfly Director's Cut,8.500000
6,Starsky & Hutch,6.500000
7,MTV's Celebrity Deathmatch,3.600000
8,Epistory - Typing Chronicles,7.700000
9,The Haunted: Hell's Reach,7.600000


In [16]:
cleaned_df['mean_user_review'] = cleaned_df['mean_user_review'].round(2)
cleaned_df = cleaned_df.rename(columns={'_id': 'Video Game'})
cleaned_df = cleaned_df.rename(columns={'mean_user_review': 'Average Rating'})

cleaned_df.head(15)

,Video Game,Average Rating
0,Oddworld: Soulstorm,6.35
1,Devil May Cry 3: Special Edition,8.60
2,Jumper: Griffin's Story,3.30
3,Cabela's Dangerous Hunts 2011,6.00
4,Happy Birthdays,7.20
5,Fatal Frame II: Crimson Butterfly Director's Cut,8.50
6,Starsky & Hutch,6.50
7,MTV's Celebrity Deathmatch,3.60
8,Epistory - Typing Chronicles,7.70
9,The Haunted: Hell's Reach,7.60


In [17]:
top_games = cleaned_df.sort_values(by='Average Rating', ascending=False)
top_games.head(15)

,Video Game,Average Rating
4161,GrimGrimoire,9.70
10418,Ghost Trick: Phantom Detective,9.70
752,Z.H.P. Unlosing Ranger vs Darkdeath Evilman,9.70
2124,Metal Torrent,9.70
9625,Tengami,9.70
3211,Diaries of a Spaceport Janitor,9.60
5719,Crystar,9.60
46,Astro's Playroom,9.30
10268,The Witcher 3: Wild Hunt,9.27
10104,The Last of Us,9.20


In [18]:
bottom_games = cleaned_df.sort_values(by='Average Rating', ascending=True)
bottom_games.head(15)

,Video Game,Average Rating
9632,Madden NFL 21,0.35
1236,Madden NFL 22,0.55
11147,Warcraft III: Reforged,0.60
8571,FIFA 20: Legacy Edition,0.70
10058,The Sims 4: Star Wars - Journey to Batuu,1.00
4033,When Ski Lifts Go Wrong,1.00
965,FIFA 21,1.07
1312,Call of Duty: Modern Warfare 3 - Defiance,1.20
7609,FIFA 20,1.30
4012,Fast & Furious: Showdown,1.30
